## Importing all necessary libararies

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from Utility.ThesisUtility import getSentcesAndTokens,iterateAndAssignGoldStandard,printPairsEvaluation,printPurity

### Constants

In [8]:
#The file path to the dataset
dataFilePath = "../Dataset/Essays_all/TelevisonMergedT1+T2"
evaluationFilePath = "../CSV Files/Clusters Evaluation results/GOLD vs Clusters - gold standards as centriodsWithTfIDf.xlsx"
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

### Reading the input(Essays) as text files and parsing it into sentences and tokens

In [9]:
sentences = getSentcesAndTokens(dataFilePath,False)

## Reading the gold standards and vectorizing them
df_ref = pd.read_excel("../CSV Files/Clusters Evaluation results/new/Examples Mapping.xlsx")

### building the sentences' vectors using TF-IDF vectorizer

In [10]:
# vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
# vectorizer.fit_transform(sentences)
# vectors = vectorizer.transform(sentences)

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")


Batches:   0%|          | 0/1210 [00:00<?, ?it/s]

In [ ]:
listExamplesVectoros =[]
for i in range(1,27):
    intern = []
    for s in df_ref[i]:
        if (pd.isna(s)):
            continue
        else:
            intern.append(s)
    intern = model.encode(intern, show_progress_bar =True, device="cuda")
    intern = np.asarray(intern)
    avg_vec = intern.mean(axis=0)
    listExamplesVectoros.append(avg_vec)

### Iterating over all the sentneces and assigning the gold standard with the highest cosin similarity

In [ ]:
iterateAndAssignGoldStandard(corpus_embeddings,sentences,listExamplesVectoros[0:25],"Sbert With Centriods2.xlsx",False)

In [28]:
len(listExamplesVectoros[0:25])

25

### Clusters Evaluation

In [ ]:
printPairsEvaluation(evaluationFilePath,"excel")

printPurity(evaluationFilePath,"excel")